In [ ]:
import pandas as pd
import pysam
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import gzip
import scipy.io
from scipy.sparse import csr_matrix

In [ ]:
vcfile = pd.read_table('xSDS_variants.txt')

In [ ]:
vcfile['Transition'] = vcfile['Ref']+'>'+vcfile['Alt']

In [ ]:
base_trans = vcfile.Transition.value_counts().iloc[:12]

In [ ]:
vcfile.Transition.value_counts().head(40)

In [ ]:
vcfile = vcfile[vcfile.Transition.isin(base_trans.index)].copy()

In [ ]:
vcfile['var_name'] = vcfile['Chr']+ '_'+ vcfile['Pos'].astype('str') + '_' + vcfile.Transition

In [ ]:
vcfile = vcfile.set_index('var_name')

In [ ]:
vcfile

In [ ]:
bam_file = '/n/scratch/users/m/meb521/153_A22KHFYLT3/xBO153_merge/xBO153_merge_markdup_piped.bam'
white_list = pd.read_csv('/n/scratch/users/m/meb521/153_A22KHFYLT3/xBO153_merge/xBO153_merge_whitelist.csv').tenx_whitelist
bc_idx_dict = dict(zip(white_list.tolist(),white_list.index))

rows_idx = []
cols_idx = []
row_col_values = []

for ii in range(len(vcfile)):
    chrom = vcfile.iloc[ii]['Chr']
    start = vcfile.iloc[ii]['Pos']-1
    end = start+1
    ref=vcfile.iloc[ii]['Ref']
    alt=vcfile.iloc[ii]['Alt']
    
    if ii%10==0:
        print(ii,vcfile.iloc[ii].name)
    samfile = pysam.AlignmentFile(bam_file, "rb" )
    for pileupcolumn in samfile.pileup(chrom, start, end, min_base_quality=0, max_depth=1000000, truncate=True): #,

        all_bases = []
        for pileupread in pileupcolumn.pileups:
            if not pileupread.is_del and not pileupread.is_refskip:

                line = [pileupread.alignment.get_tag('CB'),
                        pileupread.alignment.query_sequence[pileupread.query_position],
                        pileupread.alignment.query_qualities[pileupread.query_position]]
                all_bases.append(line)

    df = pd.DataFrame(all_bases)

    df = df[ (df[1].isin([ref,alt])) & (df[0].isin(bc_idx_dict))].copy()
    all_bcs = df[0].unique()

    df = df.set_index(0)

    vafs = []
    for bc in all_bcs:
        list_var = df.loc[bc][1]
        if len(list_var)>1:
            variants = ''.join(df.loc[bc][1].to_list())
        else:
            variants = df.loc[bc][1]
        vaf = variants.count(alt)/len(variants)
        vafs.append(vaf+1)
    
    col_vals = (np.ones(len(vafs), dtype=int) * ii).tolist()
    row_vals = [bc_idx_dict[key] for key in all_bcs]

    cols_idx.extend(col_vals)  # genes
    rows_idx.extend(row_vals)  # cells

    row_col_values.extend(vafs)
    
shape = (len(bc_idx_dict), len(vcfile))

csr = csr_matrix((row_col_values, (rows_idx, cols_idx)), shape=shape, dtype="float32")

csr_file = f"variants_colon.mtx.gz"

with gzip.open(csr_file, "wb") as out:
    scipy.io.mmwrite(out, csr)